In [35]:
dialog = open('C:/src/ML/theropy_bot/theropy_conversations/dialog.txt', encoding= 'utf-8', errors='ignore').read().split('\n')
conv = open('C:/src/ML/theropy_bot/theropy_conversations/conv.txt', encoding= 'utf-8', errors='ignore').read().split('\n')


In [36]:
exchn = []
for conver in conv:
    exchn.append(conver[1:-1].replace("'", " ").replace(",","").split())

In [37]:
diag = {}
for line in dialog:
    diag[line.split(' + ')[0]] = line.split(' + ')[-1]

In [38]:
questions = []
answers = []

In [39]:
for conver in exchn:
    for i in range(len(conver) - 1):
        questions.append(diag[conver[i]])
        answers.append(diag[conver[i+1]])

In [40]:
sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 15:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])

In [41]:
import re

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

In [42]:
clean_ques = []
clean_ans = []

for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))

In [43]:
clean_ans=clean_ans[:30000]
clean_ques=clean_ques[:30000]

In [44]:
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [45]:
thresh = 2

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1

In [46]:

for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'

In [47]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

In [48]:
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

## delete

### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}







encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)








from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 15, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 15, padding='post', truncating='post')




decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 15, padding='post', truncating='post')



from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [50]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input


enc_inp = Input(shape=(15, ))
dec_inp = Input(shape=(15, ))


VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=15,
                  trainable=True                  
                  )


enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]



dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)




model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=15)

Epoch 1/15
80/80 [==============================] - 35s 345ms/step - loss: 3.8198 - acc: 0.4232
Epoch 2/15
80/80 [==============================] - 30s 372ms/step - loss: 3.0554 - acc: 0.4588
Epoch 3/15
80/80 [==============================] - 29s 364ms/step - loss: 2.8942 - acc: 0.4837
Epoch 4/15
80/80 [==============================] - 30s 376ms/step - loss: 2.8101 - acc: 0.4976
Epoch 5/15
80/80 [==============================] - 30s 368ms/step - loss: 2.7340 - acc: 0.5070
Epoch 6/15
80/80 [==============================] - 29s 365ms/step - loss: 2.6581 - acc: 0.5147
Epoch 7/15
80/80 [==============================] - 32s 402ms/step - loss: 2.5825 - acc: 0.5264
Epoch 8/15
80/80 [==============================] - 25s 317ms/step - loss: 2.5141 - acc: 0.5325
Epoch 9/15
80/80 [==============================] - 24s 305ms/step - loss: 2.4522 - acc: 0.5383
Epoch 10/15
80/80 [==============================] - 24s 294ms/step - loss: 2.3958 - acc: 0.5445
Epoch 11/15
80/80 [====================

In [51]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


enc_model = Model([enc_inp], enc_states)



# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)

In [58]:
import numpy as np


from tensorflow.keras.preprocessing.sequence import pad_sequences

print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 15, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 15:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================")  

##########################################
#       start chatting ver. 1.0          #
##########################################
1/1 [==============================] - 0s 50ms/step
chatbot attention :  i am sorry to hear that is <OUT> that is <OUT> to <OUT> 
1/1 [==============================] - 0s 30ms/step
chatbot attention :  i am sorry to hear that is <OUT> that is <OUT> to <OUT> 
